# warp

In [ ]:
from skimage.transform import PiecewiseAffineTransform, warp
from scipy.interpolate import Rbf
# rbf코드 참고 : https://relate.cs.illinois.edu/course/cs357-f15/file-version/03473f64afb954c74c02e8988f518de3eddf49a4/media/interp/2D%20interpolation%20with%20RBF.html
class PointsRBF: # N 차원 분산 데이터에서 M 차원 영역으로
    def __init__(self, src, dst):
        xsrc = src[:,0]
        ysrc = src[:,1]
        xdst = dst[:,0]
        ydst = dst[:,1]
        self.rbf_x = Rbf( xsrc, ysrc, xdst) #ND 분산 데이터에서 MD 도메인으로 함수의 방사형 기저 함수 보간을위한 클래스
        self.rbf_y = Rbf( xsrc, ysrc, ydst)

    def __call__(self, xy):
        x = xy[:,0]
        y = xy[:,1]
        xdst = self.rbf_x(x,y)
        ydst = self.rbf_y(x,y)
        return np.transpose( [xdst,ydst] )
    
    
def warpRBF(image, src, dst): # warp : 주어진 좌표변환에 따라 이미지 왜곡
    prbf = PointsRBF(dst, src)
    warped = warp(image, prbf) 
    #warp(image, inverse_map) -> 출력 이미지의 좌표를 입력 이미지의 해당 좌표로 변환하는 역 좌표 맵 -> 이걸 PointsRBF로 구현
    warped = 255*warped                         # 0..1 => 0..255
    warped = warped.astype(np.uint8)            # convert from float64 to uint8
    return warped

In [ ]:
### from and to coordintes
coords_from_A = self.df_A_test.loc[files_A].to_numpy().reshape(20,2) #df_A_test는 원본이미지의 랜드마크 좌표들
coords_to_A = fake_B_recon.reshape(20,2)
coords_from_B = self.df_B_test.loc[files_B].to_numpy().reshape(20,2) 
coords_to_B = fake_A_recon.reshape(20,2)

# cocatenate boundary points  
coords_from_A = add_boundary_coords(coords_from_A) # 사진모서리들 경계좌표 추가
coords_to_A = add_boundary_coords(coords_to_A)
coords_from_B = add_boundary_coords(coords_from_B)
coords_to_B = add_boundary_coords(coords_to_B)

### load and warp image
fake_image_A = warpRBF(image_A, coords_from_A, coords_to_A) #(원본이미지, 원본이미지 랜드마크좌표, 냐가 바꾸고싶은 랜드마크 좌표)
fake_image_B = warpRBF(image_B, coords_from_B, coords_to_B)